In [1]:
import random
import numpy as np
import time

In [2]:
def sim_all_zero(n, shots = 8192, p = 0.05):
    hist = dict()
    for _ in range(shots):
        all_zero = ["0" for _ in range(n)]
        for i in range(n):
            if random.uniform(0, 1) < p:
                all_zero[n - 1 - i] = "1"
        state = "".join(all_zero)
        if state in hist:
            hist[state] += 1
        else:
            hist[state] = 1
    return hist

In [3]:
def sim_all_one(n, shots = 8192, p = 0.05):
    hist = dict()
    for _ in range(shots):
        all_one = ["1" for _ in range(n)]
        for i in range(n):
            if random.uniform(0, 1) < p:
                all_one[n - 1 - i] = "0"
        state = "".join(all_one)
        if state in hist:
            hist[state] += 1
        else:
            hist[state] = 1
    return hist

In [4]:
def reduce_hist(hist, poses):
    n = len(poses)
    ret_hist = {format(i, "0" + str(n) + "b"): 0 for i in range(1 << n)}
    for k, v in hist.items():
        pos = ""
        for i in range(n):
            pos += k[poses[i]]
        ret_hist[pos] += v
    return ret_hist

In [5]:
def make_cal_matrices(n, zero_hist, one_hist, shots = 8192):
    cal_matrices = []
    for i in range(n)[::-1]:
        cal_matrix = [[0,0],[0,0]]
        reduced_zero_hist = reduce_hist(zero_hist, [i])
        reduced_one_hist = reduce_hist(one_hist, [i])
        cal_matrix[0][0] = reduced_zero_hist["0"] / shots
        cal_matrix[0][1] = reduced_one_hist["0"] / shots
        cal_matrix[1][0] = reduced_zero_hist["1"] / shots
        cal_matrix[1][1] = reduced_one_hist["1"] / shots
        cal_matrices.append(cal_matrix)
    return np.array(cal_matrices)

In [6]:
shots = 1 << 10
max_iter = 10
min_size = 1
max_size = 65

In [7]:
raw_hist_list_list = []
cal_matrices_list_list = []
for num_qubits in range(min_size, max_size + 1):
    
    t1 = time.perf_counter()
            
    raw_hist_list = []
    cal_matrices_list = []
    for i in range(max_iter):
        raw_hist_list.append(sim_all_zero(num_qubits, shots = shots, p = 0.05))

        zero_hist = sim_all_zero(num_qubits, shots = shots, p = 0.05)
        one_hist = sim_all_one(num_qubits, shots = shots, p = 0.05)
        cal_matrices_list.append(make_cal_matrices(num_qubits, zero_hist, one_hist, shots))
    
    raw_hist_list_list.append(raw_hist_list)
    cal_matrices_list_list.append(cal_matrices_list)
    
    t2 = time.perf_counter()
    
    print("size", num_qubits, "finished (t = ", t2 - t1, "s)")

size 1 finished (t =  0.034481334999998836 s)
size 2 finished (t =  0.04985326999999984 s)
size 3 finished (t =  0.056131533000002065 s)
size 4 finished (t =  0.07259599799999705 s)
size 5 finished (t =  0.0856654110000008 s)
size 6 finished (t =  0.09620860100000073 s)
size 7 finished (t =  0.1082703510000016 s)
size 8 finished (t =  0.11628293700000114 s)
size 9 finished (t =  0.12635189100000233 s)
size 10 finished (t =  0.1432778980000009 s)
size 11 finished (t =  0.15359566100000066 s)
size 12 finished (t =  0.17568594399999782 s)
size 13 finished (t =  0.17280379900000042 s)
size 14 finished (t =  0.1878377649999976 s)
size 15 finished (t =  0.2083159909999992 s)
size 16 finished (t =  0.20668289299999998 s)
size 17 finished (t =  0.23200891600000162 s)
size 18 finished (t =  0.24670776299999986 s)
size 19 finished (t =  0.2580926639999994 s)
size 20 finished (t =  0.27749564800000215 s)
size 21 finished (t =  0.29332727899999966 s)
size 22 finished (t =  0.30966453699999974 s)
s

In [8]:
import pickle
with open("pkls/raw_hist_list_list.pkl", "wb") as f:
    pickle.dump(raw_hist_list_list, f)
with open("pkls/cal_matrices_list_list.pkl", "wb") as f:
    pickle.dump(cal_matrices_list_list, f)